In [11]:
import pandas as pd
import Levenshtein

In [ ]:
df_house = pd.read_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/housingquery.csv', dtype={'PrimaryID': str})

In [ ]:
pd.options.display.max_colwidth = 300
pd.options.display.max_rows = 300

## Read and format cdbg data
Take the pr03 reports and convert them into a dataframe

In [1]:
#Read all pr03 csv files
lines = []
for i in range(2007, 2018):
    file = '/home/mirabel/Dropbox (GaTech)/CDS-2019-AlbanyHub/Raw-Data/Housing/Pr03_2007/cdbg_' + str(i) + '.csv'
    with open(file, 'r') as f:
        lines.extend(f.readlines())

In [2]:
length = len(lines) #Each recored in lines is a new line in the pr03 csv

### Most of this code is just cleaning and splitting to find relevant info. 

variable names and such will be explained below

In [3]:
# split each project into separate parcel
start = 0
end = 0
parcels = []
i = 0
while i < length:
    try:
        start = end
        i += 1
        while "PGM Year" not in lines[i]: #PGM year is the start of a new project entry
            i += 1
        end = i
        parcel = lines[start:end]
        parcels.append(parcel)
    except:
        print("done")
        break

done


In [4]:
parcels[8]

['PGM Year:,,,,2006,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Project:,,,,0006 - EMERGENCY REPAIRS                                           ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'IDIS Activity:,,,,505 - 2600 LINDSEY AVENUE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 ',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Status:,,,,Canceled 8/30/2007 12:00:00 AM,,,,,,,,,,,,,,,,,,,,,Objective:,,,,,Provide decent affordable housing,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Location:,,,,"2600 Lindsey Ave                                                                                             Albany, GA  31705-2100",,,,,,,,,,,,,,,,,,,,,Outcome:,,,,,Affordability,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 ',,,,,,,,,,,,,,,,,,,,,,,,,Matrix Code:,,,,,Rehab; Single-Unit Residential (14A),,,,,,,,,,,National Objective:,,,,,,,,,,,LMH,,,,,\n',
 ',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Initial Funding Date:,,,,,,,,,,,08/20/2007,,,,

## Fields that we want to pull

In [5]:
# split each parcel into smaller bits of data with relevant info
condensedparcels = []
for parcel in parcels:
    condensed = []
    for i in range(len(parcel)):
        if "Project" in parcel[i]:
            condensed.append(parcel[i])
        if "IDIS Activity" in parcel[i]:
            condensed.append(parcel[i])
        if "Status" in parcel[i]:
            condensed.append(parcel[i])
        if "Location" in parcel[i]:
            condensed.append(parcel[i])
        if "Description" in parcel[i]:
            condensed.append("Description:" + parcel[i+1])
    #for line in parcel:
    #    if "IDIS Activity" in line:
    #        condensed.append(line)
    #    if "Status" in line:
    #        condensed.append(line)
    #    if "Location" in line:
    #        condensed.append(line)
    if len(condensed) > 0:
        condensedparcels.append(condensed)

In [6]:
condensedparcels[8]

['Project:,,,,0006 - EMERGENCY REPAIRS                                           ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'IDIS Activity:,,,,506 - 916 CEDAR AVENUE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Status:,,,,Completed 9/7/2007 12:00:00 AM,,,,,,,,,,,,,,,,,,,,,Objective:,,,,,Provide decent affordable housing,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Location:,,,,"916 Cedar Ave                                                                                             Albany, GA  31701-2945",,,,,,,,,,,,,,,,,,,,,Outcome:,,,,,Affordability,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Description:PERFORM EMERGENCY REPAIRS TO A SINGLE-FAMILY OWNER-OCCUPIED UNIT.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n']

#### formatting

In [7]:
# format to a parseable piece
totalformatted = []
for i in range(len(condensedparcels)):
    formatted = []
    #0: Project type
    proj = condensedparcels[i][0].strip("\n").strip(",").split(",")
    formatted.append(proj[0] + ":" + proj[-1])
    #1: Idis ID
    idis = condensedparcels[i][1].split("-")[0].replace(",","").replace('"',"")
    #2: status completed/open and the date
    status = condensedparcels[i][2].replace(",","").split()[0:2]
    #3: location of project
    location = condensedparcels[i][3].replace(",","").split('"')[0:2]
    formatted.append(idis.strip())
    formatted.append(status[0] + ":" + status[1])
    #4: Decription
    desc = condensedparcels[i][4].strip("\n").strip(",")
    try:
        formatted.append(location[0] + location[1])
    except:
        firstindex = 9# Case: no location listed
        secondindex = location[0].find("Outcome")
        #secondindex = location[0].split(":")[1].find("Outcome")
        #formatted.append(location[0].split(":")[1])
        formatted.append("Location:" + location[0][firstindex:secondindex])
        print("location " + location[0] + " not found")
    formatted.append(desc)
    totalformatted.append(formatted)

location Status:Completed 10/31/2010 12:00:00 AMObjective:Create suitable living environments
 not found
location Status:Completed 11/20/2013 12:00:00 AMObjective:Create suitable living environments
 not found
location Status:Completed 10/31/2010 12:00:00 AMObjective:Create suitable living environments
 not found
location Status:Completed 11/20/2013 12:00:00 AMObjective:Create suitable living environments
 not found
location Status:Completed 11/20/2013 12:00:00 AMObjective:Create suitable living environments
 not found
location Location:Address SuppressedOutcome:Availability/accessibility
 not found
location Status:Completed 11/20/2013 12:00:00 AMObjective:Create suitable living environments
 not found
location Location:Address SuppressedOutcome:Availability/accessibility
 not found
location Status:Completed 11/20/2013 12:00:00 AMObjective:Create suitable living environments
 not found
location Location:Address SuppressedOutcome:Availability/accessibility
 not found
location Status:Com

In [8]:
totalformatted[314] #Example reformatted list

['Project::0015 - PUBLIC SERVICE',
 'IDIS Activity:774',
 'Status:Completed:10/31/2010',
 'Location:PO Box 44   Albany GA  31702-0044',
 'Description:To expand the service of assessing individuals served to determine needs; and to enhance the services of the soup kitchen.']

In [9]:
# slightly more formatting
proj_list = []
idisno_list = []
status_list = []
status_date_list = []
location_list = []
desc_list = []
for project in totalformatted:
    proj_list.append(project[0].split(":")[-1]) #Part after dash - project description
    idisno_list.append(project[1].split(":")[1]) #Part before dash - idis number
    status_full = project[2].split(":")
    status_list.append(status_full[1])#Completed/open/cancelled
    status_date_list.append(status_full[2]) #Date of status
    location_list.append(project[3].split(":")[1].strip()) #address
    desc_list.append(project[4].split(":")[1])#description

In [12]:
# renaming columns
df = pd.DataFrame({'Project': proj_list, 'Activity': idisno_list, 'Status': status_list, 
                   'Status Date': status_date_list, 'Address': location_list, 'Description': desc_list})

# Start: df is name of dataframe.

### descriptions may need to be cleaned up a bit more also. Can pull in more fields if necessary from the pr03s. The thing that may be useful is pulling in the objective.

In [13]:
df.head()

,Project,Activity,Status,Status Date,Address,Description
0,0009 - PUBLIC SERVICE ...,445,Completed,6/30/2006,230 S Jackson St ...,"""TO HELP THE HOMELESS IN ALBANY, GEORGIA"""
1,0006 - HOUSING,466,Canceled,1/10/2012,SANDS DRIVE ALBANY GA 31705,"""THE ACQUISITION OF PROPERTY FOR THE DEVELOPME..."
2,0001 - ADMINISTRATION ...,473,Completed,6/30/2007,230 S Jackson St SUITE 315 ...,"""THE OPERATION OF THE CDBG PROGRAM."
3,0002 - SECTION 108 LOAN ...,475,Completed,8/25/2007,230 S Jackson St SUITE 315 ...,"""THE SECTION 108 PAYMENT FOR THE LOAN RELATED ..."
4,0002 - SECTION 108 LOAN ...,476,Completed,8/25/2007,230 S Jackson St SUITE 315 ...,THE REPAYMENT OF A SECTION 108 LOAN FOR THE MC...


In [14]:
df[df['Address'].str.contains("OGLETHORPE")]

,Project,Activity,Status,Status Date,Address,Description


In [16]:
df[df['Description'].str.upper().str.contains("WINDOWS")]

,Project,Activity,Status,Status Date,Address,Description
402,0005 - Energy Efficiency,821,Completed,9/27/2010,3005 Harvest Ln Albany GA 31721-4514,"""Energy Efficiency measures were installed to ..."
403,0005 - Energy Efficiency,822,Canceled,2/10/2011,103 Almond Dr Albany GA 31705-2528,Energy Efficiency measures were installed to i...
404,0005 - Energy Efficiency,823,Completed,3/2/2011,706 Easttown Rd Albany GA 31705-1214,"""Energy Efficiency measures were installed to ..."
405,0005 - Energy Efficiency,824,Completed,9/30/2010,919 W Lincoln Ave Albany GA 31701-2919,"""Energy Efficiency measures were installed to ..."
406,0005 - Energy Efficiency,825,Completed,2/14/2011,2004 Colquitt Ave Albany GA 31707-5129,"""Energy Efficiency measures were installed to ..."
408,0005 - Energy Efficiency,827,Completed,2/14/2011,2514 Whispering Pines Rd Albany GA 31707-2234,"""Energy Efficiency measures were installed to ..."
409,0005 - Energy Efficiency,829,Completed,2/2/2011,1530 E Broad Ave Albany GA 31705-2210,Energy Efficiency measures were installed to i...
410,0005 - Energy Efficiency,830,Completed,2/14/2011,2230 S Washington St Albany GA 31701-3735,"""Energy Efficiency measures were installed to ..."
413,0005 - Energy Efficiency,833,Completed,4/28/2011,1105 Jordan St Albany GA 31705-1371,Energy Efficiency measures were installed to i...
415,0005 - Energy Efficiency,835,Canceled,2/9/2011,402 Bush St Albany GA 31705-4004,Energy Efficiency measures were installed to i...


## Everything below is pulling in HOME files and standardizing addresses. Would recommend copying in everything between start and here and copying into a separate file.

In [17]:
# s_addresses contains all the addresses in the pr03 files
s_addresses = df['Address']
for i in range(len(s_addresses)):
    words = s_addresses[i].split()
    index = 0
    for j in range(len(words)):
        if words[j] == "Albany" or words[j] == "ALBANY":
            index = j
    words = words[0:index]
    s_addresses[i] = words

In [18]:
s_addresses.head()

0                [230, S, Jackson, St]
1                       [SANDS, DRIVE]
2    [230, S, Jackson, St, SUITE, 315]
3    [230, S, Jackson, St, SUITE, 315]
4    [230, S, Jackson, St, SUITE, 315]
Name: Address, dtype: object

In [ ]:
# some format/capitalization/casting for the addresses
for i in range(len(s_addresses)):
    for j in range(len(s_addresses[i])):
        if not s_addresses[i][j].isnumeric():
            s_addresses[i][j] = s_addresses[i][j].upper()

In [ ]:
s_addresses.head()

In [ ]:
df_house = df_house[df_house['Program'] == 'CDBG']

In [ ]:
df.shape

In [ ]:
# have to reformat address so that we pd can hash to drop duplicates
for i in range(len(df)):
    addr = df['Address'][i]
    word = ""
    for pt in addr:
        word += pt + " "
    word = word.strip()
    df['Address'][i] = word

In [ ]:
df = df.drop_duplicates()

In [ ]:
# might help with matching
def lev_list(address_list, address,i):
    temp_list = []
    temp_list_u = []
    for u_address in address_list:
        if Levenshtein.distance(address, u_address.upper()) <= i:
            temp_list.append(address)
            temp_list_u.append(u_address.upper())
    temp_list_series = pd.Series(temp_list)
    temp_list_u_series = pd.Series(temp_list_u)
    return temp_list_u_series

#### df is the dataframe with completed date
#### df_house is the dataframe with housing projects

In [ ]:
df_house.index = range(len(df_house))

In [ ]:
dates = []
addresses = []
for i in range(len(df_house)):
    completed_addresses = df[str(df_house['IDISActivityID'][i]) == df['Activity']]
    if len(completed_addresses) == 1:
        completed_addresses.index = range(len(completed_addresses))
        date = completed_addresses['Status Date'][0]
        addr = completed_addresses['Address'][0] # for checking purposes
        if completed_addresses['Status'][0] == "Open":
            dates.append("Continuing")
        else:
            dates.append(date)
        addresses.append(addr)
    elif len(completed_addresses) == 0:
        dates.append("NA")
    elif len(completed_addresses) == 2:
        print("multiple")

In [ ]:
addresses[2]

In [ ]:
dates = pd.Series(dates)

In [ ]:
dates[0]

In [ ]:
df_house.head()

In [ ]:
dates[57]

In [ ]:
df[df['Address'] == '610 W LINCOLN AVE']

In [ ]:
df_house[df_house['IDISActivityID'] == 1235]

In [ ]:
parcels[836]

In [ ]:
df.shape

In [ ]:
dates.shape

In [ ]:
df_house.head()

In [ ]:
df_house.iloc[260]

In [ ]:
addresses[260]

In [ ]:
dates[dates == "NA"]

In [ ]:
dates[dates == "Continuing"]

In [ ]:
df_house.iloc[26]

In [ ]:
df[df['Activity'] == 1235]

#### Reload in dataset to fix home grants with different formatting

In [ ]:
df_house_home = pd.read_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/housingquery.csv', dtype={'PrimaryID': str})

In [ ]:
df_home = pd.read_csv('/Users/william/Documents/Summer2019/AlbanyHousing/Raw Data/CED/parse/homefull.csv')

In [ ]:
df_home.tail()

In [ ]:
df_home['Activity Address'] = df_home['Activity Address'].str.split(" , ", expand = True)[0]

In [ ]:
df_home.tail()

In [ ]:
df_house_home = df_house_home[df_house_home['Program'] == "HOME"]

In [ ]:
df_house_home.index = range(len(df_house_home))

In [ ]:
df_house_home['IDISActivityID'][0]

In [ ]:
df_home.head()

In [ ]:
dates_home = []
addresses_home = []
for i in range(len(df_house_home)):
    completed_addresses_home = df_home[df_house_home['IDISActivityID'][i] == df_home['IDIS Activity']]
    if len(completed_addresses_home) == 1:
        completed_addresses_home.index = range(len(completed_addresses_home))
        date_home = completed_addresses_home['Status Date'][0]
        addr_home = completed_addresses_home['Activity Address'][0] # for checking purposes
        if completed_addresses_home['Activity Status'][0] == "Open":
            dates_home.append("Continuing")
        else:
            dates_home.append(date_home)
        addresses_home.append(addr_home)
    elif len(completed_addresses_home) == 0:
        dates_home.append("NA")
    elif len(completed_addresses_home) == 2:
        print("multiple")

In [ ]:
dates_home

In [ ]:
dates_home[1]

In [ ]:
addresses[1]

In [ ]:
df_house_home.loc[0]